<a href="https://colab.research.google.com/github/Bayzid03/QNA-ChatBot-with-Chat-History/blob/main/Fine_Tuning_DeepSeek_on_Custom_DS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets sympy wandb
!pip install --no-cache-dir bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 256.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 228.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 255.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 258.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 381.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 277.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 234.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 161.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 198.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 151.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 176.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    U

In [ ]:
!pip install transformers

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to('cuda')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/3.07k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/679 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [ ]:
import json

# Load the standard JSON file
with open("space_qa_dataset.json", "r", encoding="utf-8") as f:
    data = json.load(f)
    samples = data["samples"]  # Extract the list of Q&A pairs

print(f"Loaded {len(samples)} samples.")
# Example: Print the first sample
print(samples[0])

Loaded 71 samples.
{'prompt': 'What is the closest planet to the Sun?', 'completion': 'The closest planet to the Sun is Mercury.'}


In [ ]:
!wandb login

In [ ]:
import wandb
wandb.init(
    project="Fine Tuning LLM",
    config={
        "learning rate": 5e-5,
        "architecture": "DeepSeek-R1-Distill-Qwen-1.5B",
        "dataset": "space_qa_dataset.json",
        "epochs": 2,
    }
)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json
from datasets import Dataset

# 1. Load your JSON file manually
with open("/content/space_qa_dataset.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# 2. Extract the samples list
samples = data["samples"]

# 3. Create a Dataset object directly
dataset = Dataset.from_list(samples)

# 4. Now split the dataset
train_test_split = dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split["train"]
test_dataset = train_test_split["test"]

print(f"Total samples: {len(dataset)}")
print(f"Train samples: {len(train_dataset)}")
print(f"Test samples: {len(test_dataset)}")

Total samples: 71
Train samples: 56
Test samples: 15


In [ ]:
def tokenize_function(examples):
    """Tokenize prompt-completion pairs for instruction fine-tuning."""
    texts = [
        f"Question: {prompt}\\nAnswer: {completion}"
        for prompt, completion in zip(examples["prompt"], examples["completion"])
    ]

    # Tokenize with truncation and padding
    tokenized = tokenizer(
        texts,
        truncation=True,
        max_length=512,
        padding="max_length",
        return_tensors="pt"  # Return PyTorch tensors
    )

    # Create labels (simply use input_ids, DataCollatorForLanguageModeling will handle shifting)
    tokenized["labels"] = tokenized["input_ids"] # Corrected line

    return tokenized

# Apply to both datasets
tokenized_train_dataset = train_dataset.map(
    tokenize_function,
    batched=True,
    batch_size=32,  # Process in batches for efficiency
    remove_columns=["prompt", "completion"]  # Remove original columns
)

tokenized_eval_dataset = test_dataset.map(
    tokenize_function,
    batched=True,
    batch_size=32,
    remove_columns=["prompt", "completion"]
)

Map:   0%|          | 0/56 [00:00<?, ? examples/s]

Map:   0%|          | 0/15 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForCausalLM, BitsAndBytesConfig
import torch

# Configure 8-bit quantization
quantization_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_threshold=6.0,  # Default threshold for outlier detection
    llm_int8_skip_modules=None,  # Optional: list of modules to not quantize
    torch_dtype=torch.float16  # Recommended for compatibility
)

# Load model with quantization
model = AutoModelForCausalLM.from_pretrained(
    "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B",
    quantization_config=quantization_config,
    device_map="auto",  # Automatic device placement (GPU/CPU)
    trust_remote_code=True  # Required for some models
)

In [ ]:
from peft import get_peft_model, LoraConfig, TaskType

# Optimal LoRA configuration for DeepSeek-R1-Distill-Qwen-1.5B
lora_config = LoraConfig(
    r=8,                  # Rank (lower = less memory, but may reduce quality)
    lora_alpha=16,        # Scaling factor (typically 2x rank)
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],  # Key layers to target
    lora_dropout=0.05,    # Regularization to prevent overfitting
    bias="none",          # Don't train bias params (saves memory)
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False   # Important for training!
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 2,179,072 || all params: 1,779,267,072 || trainable%: 0.1225


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./deepseek_finetuned",  # Directory to save checkpoints
    num_train_epochs=3,                # Reduced from 50 (avoids overfitting)
    per_device_train_batch_size=4,     # Increased from 2 (if GPU memory allows)
    gradient_accumulation_steps=4,     # Reduced from 16 (better balance)
    optim="adamw_torch_fused",         # Optimizer (faster than default)
    fp16=True,                         # Mixed-precision training
    bf16=False,                        # Disable if not on Ampere+ GPU
    logging_steps=10,
    save_steps=200,                    # Save less frequently
    # Replace 'evaluation_strategy' with 'eval_strategy'
    eval_strategy="steps",
    eval_steps=100,                    # Evaluate every 100 steps
    learning_rate=2e-5,                # Lower LR for fine-tuning
    warmup_ratio=0.03,                 # Gradual LR warmup
    weight_decay=0.01,                 # Regularization
    lr_scheduler_type="cosine",        # Better than linear
    logging_dir="./logs",
    report_to="wandb",                 # Weights & Biases logging
    run_name="DeepSeek_LoRA_FineTune", # Experiment name
    save_total_limit=2,                # Max checkpoint saves
    gradient_checkpointing=True,       # Saves VRAM (enable if OOM)
    dataloader_num_workers=1,         # Faster data loading
    remove_unused_columns=True,        # Saves memory
)

In [ ]:
from transformers import Trainer, DataCollatorForLanguageModeling

# Create the data collator:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,  # Changed 'tokenized' to 'tokenizer'
    mlm=False  # Set to False since you're not doing Masked Language Modeling
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset,
    data_collator=data_collator,  # Add this line
)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
trainer.train()